# AI-Powered Edge-Case Test Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/03_advanced_workflows/ai_contract_enrichment/ai_edge_cases_demo.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/03_advanced_workflows/ai_contract_enrichment/ai_edge_cases_demo.ipynb)


## Business Scenario

Your team has written quality rules for the orders pipeline:

```yaml
quality:
  row_rules:
    - sql: "status IN ('pending', 'confirmed', 'shipped', 'delivered', 'cancelled')"
    - sql: "amount > 0"
    - sql: "customer_email LIKE '%@%.%'"
```

But your tests only use **happy-path data**. In production, you'll encounter:

- Emails like `user+tag@sub.domain.co.uk` — does your rule handle that?
- Amounts of exactly `0.00` — is `> 0` correct, or should it be `>= 0`?
- Status values with wrong casing: `Confirmed` vs `confirmed`
- Unicode in customer names: `José García`, `田中太郎`
- SQL injection in notes: `'; DROP TABLE orders --`
- Dates in the far future: `2099-12-31`

The LLM suggests edge cases that a human QA engineer would think of — but in seconds.

## Prerequisites

```bash
pip install "lakelogic[all]" openai
export OPENAI_API_KEY=sk-...
```

## Step 1: Setup

In [ ]:
from pathlib import Path
import yaml
from lakelogic.core.generator import DataGenerator

# Resolve the example directory
def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        candidates = [
            base / name,
            base / "examples" / "03_advanced_workflows" / name,
        ]
        for candidate in candidates:
            if candidate.exists():
                return candidate
    return cwd / name

base_path = resolve_example_dir("ai_contract_enrichment")
contract_path = base_path / "contract_ecommerce.yaml"

print(f"Contract: {contract_path}")

## Step 2: Generate with Heuristic Bad Data (No AI)

First, let's see what LakeLogic generates **without AI**. The heuristic engine produces obviously fake values like `INVALID_ABCD` — useful for basic testing, but not realistic.

In [ ]:
print("=" * 60)
print("WITHOUT AI: Heuristic Edge Cases")
print("=" * 60)
print()

gen = DataGenerator(contract_path, seed=42)
df_heuristic = gen.generate(rows=20, invalid_ratio=0.3, output_format="polars")

print("Sample rows (heuristic bad data):")
print(df_heuristic.head(5))
print()

# Show what kind of bad data the heuristics produce
bad_statuses = [
    str(v) for v in df_heuristic["status"].to_list()
    if v is not None and str(v).startswith("INVALID_")
]
print(f"Heuristic bad statuses: {bad_statuses[:5]}")
print("  → These are obviously fake values like 'INVALID_ABCD'")
print("  → A real production bug would be 'Confirmed' (wrong case)")

## Step 3: Generate with AI Edge Cases

Now let's use the LLM to generate **realistic** edge cases. The AI reads your quality rules and suggests values that would actually appear in production — wrong casing, boundary values, format variations, encoding issues.

In [ ]:
print("=" * 60)
print("WITH AI: LLM-Suggested Edge Cases")
print("=" * 60)
print()

gen_ai = DataGenerator(contract_path, seed=42)
df_ai = gen_ai.generate(
    rows=20,
    invalid_ratio=0.3,
    output_format="polars",
    ai=True,
    # ai_provider="openai",    # default
    # ai_model="gpt-4o-mini",  # default
)

print("Sample rows (AI edge cases):")
print(df_ai.head(5))

## Step 4: Inspect the AI-Suggested Edge Cases

Let's look at exactly what the LLM suggested for each field. These are the edge-case pools that get injected into invalid rows.

In [ ]:
from lakelogic.ai.edge_case_generator import generate_edge_cases

print("=" * 60)
print("INSPECT: What Edge Cases Did the AI Suggest?")
print("=" * 60)
print()

contract_data = yaml.safe_load(contract_path.read_text(encoding="utf-8"))
fields = contract_data["model"]["fields"]
quality = contract_data.get("quality", {})

edge_pools = generate_edge_cases(
    fields,
    quality,
    dataset_name="E-Commerce Orders",
)

for field_name, cases in edge_pools.items():
    print(f"  {field_name}:")
    for case in cases[:5]:
        print(f"    → {repr(case)}")
    if len(cases) > 5:
        print(f"    ... and {len(cases) - 5} more")
    print()

## Step 5: Reuse Edge Cases Across Test Runs

Pre-computed edge cases avoid repeated LLM calls. Generate once, reuse across multiple test datasets with different seeds.

In [ ]:
print("=" * 60)
print("REUSE: Generate multiple datasets with the same edge cases")
print("=" * 60)
print()

for seed in [1, 2, 3]:
    gen = DataGenerator(contract_path, seed=seed)
    df = gen.generate(
        rows=100,
        invalid_ratio=0.15,
        output_format="polars",
        ai_edge_cases=edge_pools,  # reuse — no LLM call
    )
    print(f"  Seed {seed}: {len(df)} rows generated, ~15% with AI edge cases")

## Step 6: Validate — Run Contract Against Edge-Case Data

The real test: run the contract against AI-generated data and see if the quarantine catches the edge cases.

In [ ]:
from lakelogic import DataProcessor

print("=" * 60)
print("VALIDATE: Run contract against AI-generated edge case data")
print("=" * 60)
print()

try:
    # Generate test data with AI edge cases
    temp_path = base_path / "data" / "_temp_ai_test.parquet"
    gen = DataGenerator(contract_path, seed=42)
    df_test = gen.generate(
        rows=200,
        invalid_ratio=0.2,
        output_format="polars",
        ai_edge_cases=edge_pools,
    )
    df_test.write_parquet(temp_path)

    # Run the contract
    result = DataProcessor(
        contract=contract_path,
        engine="polars",
    ).run_source(temp_path)

    print(f"  ✅ Valid rows:      {len(result.good):,}")
    print(f"  ❌ Quarantined:     {len(result.bad):,}")
    print()
    print("  The quarantine caught the AI-generated edge cases!")
    print("  This proves your quality rules handle real-world bad data.")

    # Clean up
    temp_path.unlink(missing_ok=True)

except Exception as e:
    print(f"  ⚠ Validation step skipped: {e}")
    print("  (This is expected if you haven't set up all dependencies)")

## Summary

### Heuristic vs AI Edge Cases

| Aspect | Heuristic | AI |
| :--- | :--- | :--- |
| **Status values** | `INVALID_ABCD` | `Confirmed` (wrong case), `processing` (not in enum) |
| **Emails** | `INVALID_EMAIL` | `user@.com`, `user@localhost`, `user+tag@sub.domain.co.uk` |
| **Amounts** | `-999` | `0.00` (boundary), `0.001` (precision), `-0.01` (just below zero) |
| **Names** | `INVALID_NAME` | `José García` (unicode), `田中太郎` (CJK), `O'Brien` (apostrophe) |
| **Notes** | `null` | `'; DROP TABLE orders --` (SQL injection) |

### Business Impact

- **Test coverage**: Discovers rule gaps BEFORE production incidents
- **Reduced on-call burden**: Edge cases that would page you at 3am are caught in CI
- **Compliance confidence**: Proves quarantine handles PII violations correctly

### CLI Equivalent

```bash
lakelogic generate \
    --contract contract_ecommerce.yaml \
    --rows 1000 \
    --invalid-ratio 0.15 \
    --ai \
    --preview 10
```